Importing utils

In [ ]:
import os
from dotenv import load_dotenv
from anthropic import Anthropic

# load environment variable
load_dotenv()

# automatically looks for an "ANTHROPIC_API_KEY" environment variable
client = Anthropic()

Simple call

In [ ]:
conversation_assistant_log = []
conversation_history = []

while True:
    user_input = input("User: ")

    if user_input.lower() == "quit":
        print("Conversation ended.")
        break

    conversation_history.append({"role": "user", "content": user_input})

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        messages=conversation_history,
        system_prompt="You are a helpful assistant that can answer questions and help with tasks.",
        max_tokens=500,
        stop_sequences=[],
        temperature=0
    )

    assistant_response = response.content[0].text
    print(f"Assistant: {assistant_response}")

    # log the assistant response
    conversation_history.append({"role": "assistant", "content": assistant_response})
    conversation_assistant_log.append(f"Assistant: {assistant_response}")

Excercise 1

In [ ]:
import pprint


def generate_questions(topic="AI", number_of_questions=3):
    questions = []
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        messages=[
            {
                "role": "user",
                "content": f"Based on the topic '{topic}', can you generate {number_of_questions}question for me? Only respond with the question, no other text. Generate a number list of questions",
            }
        ],
        system="You are a helpful assistant that can answer questions and help with tasks.",
        max_tokens=500,
        stop_sequences=["4"],
        stream=True
    )
    assistant_response = response.content[0].text
    questions.append(assistant_response)
    return questions

pprint.pprint(generate_questions())

Streaming 101

In [ ]:
def measuring_ttft_streming():
    stream = client.messages.create(
        messages=[
            {
                "role": "user",
                "content": "How do large language models work?",
            }
        ],
        model="claude-3-haiku-20240307",
        max_tokens=200,
        temperature=0,
        stream=True,
    )

    for event in stream:
        if event.type == "message_start":
            print("Input tokens :",event.message.usage.input_tokens, flush=True)
        elif event.type == "content_block_delta":
            print(event.delta.text, end="", flush=True)
        elif event.type == "message_delta":
            print("\n Output tokens:",event.usage.output_tokens, flush= True)

measuring_ttft_streming()

Streaming with helper functions

In [ ]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic()


async def streaming_with_helpers():

    conversation_history = []

    while True:
        user_input = input("User :")

        if user_input.lower() == "quit":
            print("Conversation Ended")
            return conversation_history

        conversation_history.append({"role": "user", "content": user_input})

        async with client.messages.stream(
            model="claude-3-5-haiku-20241022", max_tokens=200, messages=conversation_history
        ) as stream:
            async for text in stream.text_stream:
                print(text, end="", flush=True)

        assistant_final_response = await stream.get_final_message()
        assistant_text_response = assistant_final_response.content[0].text

        conversation_history.append(
            {"role": "assistant", "content": assistant_text_response}
        )

        print("\nconversation_history \n", conversation_history)


await streaming_with_helpers()

Async Chat with color coding

In [ ]:
from anthropic import AsyncAnthropic, AsyncMessageStream

client = AsyncAnthropic()

# ANSI color codes
BLUE = "\033[94m"
GREEN = "\033[92m"
RESET = "\033[0m"

async def streaming_with_helpers():

    conversation_history = []

    while True:
        user_input = input("User :")
        print(f"{BLUE}User : {RESET}{user_input}")

        if user_input.lower() == "quit":
            print("Conversation Ended")
            return conversation_history

        conversation_history.append({"role": "user", "content": user_input})

        async with client.messages.stream(
            model="claude-3-5-haiku-20241022",
            max_tokens=200,
            messages=conversation_history,
        ) as stream:
            async for text in stream.text_stream:
                print(f"{GREEN}{text}", end="", flush=True)

        assistant_final_response = await stream.get_final_message()
        assistant_text_response = assistant_final_response.content[0].text

        conversation_history.append(
            {"role": "assistant", "content": assistant_text_response}
        )

        print("\nconversation_history \n", conversation_history)


await streaming_with_helpers()

Async with MySteam custom output events ( source code does not work )

In [ ]:
# Copy pasted code block does not work ( probably depricated ) Source : https://github.com/anthropics/courses/blob/master/anthropic_api_fundamentals/05_Streaming.ipynb

from anthropic import AsyncAnthropic, AsyncMessageStream

client = AsyncAnthropic()

green = "\033[32m"
reset = "\033[0m"


class MyStream(AsyncMessageStream):
    async def on_text(self, text, snapshot):
        # This runs only on text delta stream messages
        print(
            green + text + reset, flush=True
        )  # model generated content is printed in green

    async def on_stream_event(self, event):
        # This runs on any stream event
        print("on_event fired:", event.type)


async def streaming_events_demo():
    async with client.messages.stream(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "Generate a 5-word poem",
            }
        ],
        model="claude-3-opus-20240229",
        event_handler=MyStream,
    ) as stream:
        # Get the final accumulated message, after the stream is exhausted
        message = await stream.get_final_message()
        print("accumulated final message: ", message.to_json())


await streaming_events_demo()

Prompting with images

In [ ]:
import base64
import mimetypes


def create_image_message(image_path):
    # Open the image file in "read binary" mode
    with open(image_path, "rb") as image_file:
        # Read the contents of the image as a bytes object
        binary_data = image_file.read()

    # Encode the binary data using Base64 encoding
    base64_encoded_data = base64.b64encode(binary_data)

    # Decode base64_encoded_data from bytes to a string
    base64_string = base64_encoded_data.decode("utf-8")

    # Get the MIME type of the image based on its file extension
    mime_type, _ = mimetypes.guess_type(image_path)

    # Create the image block
    image_block = {
        "type": "image",
        "source": {"type": "base64", "media_type": mime_type, "data": base64_string},
    }

    return image_block

In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Image 1"},
            create_image_message("./test_image.jpeg"),
            {"type": "text", "text": "Where might I find this in the world?"},
        ]
    },
]

response = client.messages.create(
    model="claude-haiku-4-5-20251001",
    max_tokens=300,
    messages=messages,
)

print(response.content[0].text)

In [ ]:
response[0].content[0].text

In [ ]:
response.usage.input_tokens, response.usage.output_tokens

Image from Web (  unable to run )

In [ ]:
import base64
import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Church_of_light.jpg/1599px-Church_of_light.jpg"
image_media_type = "image/jpeg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": image_media_type,
                    "data": image_data,
                },
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]


response = client.messages.create(
    model="claude-3-5-sonnet-20240620", max_tokens=2048, messages=messages
)
print(response.content[0].text)

In [ ]:
import httpx


def get_image_dict_from_url(image_url):
    # Send a GET request to the image URL and retrieve the content
    response = httpx.get(image_url)
    image_content = response.content

    # Determine the media type of the image based on the URL extension
    # This is not a foolproof approach, but it generally works
    image_extension = image_url.split(".")[-1].lower()
    if image_extension == "jpg" or image_extension == "jpeg":
        image_media_type = "image/jpeg"
    elif image_extension == "png":
        image_media_type = "image/png"
    elif image_extension == "gif":
        image_media_type = "image/gif"
    else:
        raise ValueError("Unsupported image format")

    # Encode the image content using base64
    image_data = base64.b64encode(image_content).decode("utf-8")

    # Create the dictionary in the proper image block shape:
    image_dict = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": image_media_type,
            "data": image_data,
        },
    }

    return image_dict

In [ ]:
url1 = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Rincon_fire_truck.png/1600px-Rincon_fire_truck.png"
url2 = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Ornge_C-GYNP.jpg/1600px-Ornge_C-GYNP.jpg"

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Image 1:"},
            get_image_dict_from_url(url1),
            {"type": "text", "text": "Image 2:"},
            get_image_dict_from_url(url2),
            {"type": "text", "text": "What do these images have in common?"},
        ],
    }
]


response = client.messages.create(
    model="anthropic.claude-haiku-4-5-20251001-v1:0", max_tokens=2048, messages=messages
)
print(response.content[0].text)

Example of prompt with tags

In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./people.png"),
            {
                "type": "text",
                "text": "You have perfect vision and pay great attention to detail which makes you an expert at counting objects in images. How many people are in this picture? Some of the people may be partially obscured or cut off in the image or may only have an arm visible. Please count people even if you can only see a single body part. Before providing the answer in <answer> tags, think step by step in <thinking> tags and analyze every part of the image."
            }
        ],
    }
]

response = client.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=300,
    messages=messages,
)

print(response.content[0].text)

Structured Output

In [ ]:
response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    betas=["structured-outputs-2025-11-13"],
    messages=[
        {
            "role": "user",
            "content": "Extract the key information from this email: John Smith (john@example.com) is interested in our Enterprise plan and wants to schedule a demo for next Tuesday at 2pm.",
        }
    ],
    output_format={
        "type": "json_schema",
        "schema": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "email": {"type": "string"},
                "plan_interest": {"type": "string"},
                "demo_requested": {"type": "boolean"},
            },
            "required": ["name", "email", "plan_interest", "demo_requested"],
            "additionalProperties": False,
        },
    },
)

In [ ]:
print(response.content[0].text)

In [ ]:
response

In [ ]:
print(f"stop_reason: {response.stop_reason}")
print(f"stop_sequence: {response.stop_sequence}")
print(f"input_tokens: {response.usage.input_tokens}")
print(f"output_tokens: {response.usage.output_tokens}")
print(f"server_tool_use: {response.usage.server_tool_use}")